L'exercice pour le cours prochain est le suivant:

 - Récupérer via crawling la liste des 256 top contributors sur cette page https://gist.github.com/paulmillr/2657075
 - En utilisant l'API github (https://developer.github.com/v3/) récupérer pour chacun de ces users le nombre moyens de stars des repositories qui leur appartiennent. Pour finir, classer ces 256 contributors par leur note moyenne.

Comme l'API github dispose de restrictions d'accès (limitation du nombre de requêtes), vous aurez besoin de vous authentifier via un token: 

https://help.github.com/en/articles/creating-a-personal-access-token-for-the-command-line.


In [1]:
import requests
from bs4 import BeautifulSoup
from github import Github

ACCESS_TOKEN = 'a8f657dd208713df6db3e0a948caa089224479c4'

ImportError: cannot import name 'Github'

In [2]:
def get_soup_from_url(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    
    return soup


def get_top_github_contributors():
    url = "https://gist.github.com/paulmillr/2657075"
    
    soup = get_soup_from_url(url)

    contributors = []

    table = soup.find('tbody')
    
    for row in table.find_all('tr'):
        if (row.find('th', {'scope': 'row'}) != None):
            contributors.apppend(row.find('a').text)

    return contributors

In [6]:
# Main
# https://gist.github.com/paulmillr/2657075
def main():
    # connect to github
    g = Github(ACCESS_TOKEN)

    contributors = get_top_github_contributors()

    # initialize dictionnary
    avg_stars_per_user = {}

    # loop over top contributors
    for c, url in contributors:
        user = g.get_user(c)
        avg_stars_per_user[user.login] = 0.0

        count = 0
        # loop over all repositories
        for r in user.get_repos():
            avg_stars_per_user[user.login] += r.stargazers_count
            count += 1

        # compute the average
        if count > 0:
            avg_stars_per_user[user.login] /= count

    sorted_avg = sorted(avg_stars_per_user.items(), key=lambda kv: kv[1], reverse=True)

    for key, value in sorted_avg:
        print('{} : {:.3f}'.format(key, value))

In [7]:
%%timeit

main()

ReadTimeout: HTTPSConnectionPool(host='api.github.com', port=443): Read timed out. (read timeout=15)